This script explores the effect of temperature threshold exceedances on global GDP.

Key research questions we look at are 
 - What is a good model for relating GDP to metrics describing GMT? 
 - What is the marginial effect of cumulative exceedane depth & global GDP? 
 - What is the marginial effect of cumulative exceedane depth on country-level GDP? 

The goal of this script is to find a suitable model for answering the above questions. 
To this end we explore a variety of different linear models that all use different 
sets of predictive variables and explore different target variables descirbing future GDP. 
We assess/compare the models based on their BIC and discard any models that lead to insignificant
predictor variables.  

In [1]:
import sys
sys.path.append('/Users/schoens/Documents/Projekte/Econ/Code/v3/')

import numpy as np
import pandas as pd 
import xarray as xr
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

from pathlib import Path
import config.settings as cset

import matplotlib.pyplot as plt
import seaborn as sns

## Step 1: Data Preparation

In [2]:
# loading & preparing gdp data
gdp_df   = pd.read_csv(Path.joinpath(cset.path_GDP, 'Full Data BHM Main Result.csv')).loc[:, ['iso', 'scen', 'run', 'year', 'diff_SSP2']]
gdp_df.loc[gdp_df.scen == 'Ref1p5', 'scen'] = 'Ref'
# only keep data for year that we are interested in: 
gdp_df   = gdp_df[(gdp_df.year == cset.gdp_target_year)].copy()
# adding for compatibility with characteristics df 
gdp_df['scen_run'] = gdp_df['scen'] + '_' + gdp_df['run'].astype(str)

# loading & preparing tas_baseline data
tas_df = pd.read_csv(Path.joinpath(cset.path_GDP, '2310_countrylevel_total.csv'))
tas_df = tas_df[tas_df.year == 2015].copy()
tas_df = tas_df.rename(columns = {'run': 'scen_run', 'tas': 'tas_soc'})
tas_df = tas_df.dropna(subset=['tas_soc']).groupby('iso').filter(lambda x: x['tas_soc'].notna().all())
tas_df.loc[tas_df['scen_run'].str.contains('Ref_1p5'), 'scen_run'] = tas_df['scen_run'].str.replace('Ref_1p5', 'Ref')

# loading & preparing glmt data 
glmt_char_df = pd.read_csv(Path.joinpath(cset.path_MESMER_char, 'glmt_characteristics_thsld_1462.csv'), index_col = 0)

# merge datasets into a single regression dataset
regression_df = gdp_df.loc[:, ['iso', 'scen_run', 'diff_SSP2']].merge(glmt_char_df, left_on='scen_run', right_index=True)
regression_df = regression_df.merge(tas_df[['iso', 'scen_run', 'tas_soc']], on=['iso', 'scen_run'], how='left')
#   - add baseline temperature values
regression_df['tas_baseline'] = regression_df.groupby('iso')['tas_soc'].transform('mean')
regression_df                 = regression_df.sort_values(['iso'])

# clear up storage
del gdp_df, glmt_char_df, tas_df

## Step 2: Constructing different models and selecting by BIC

In [3]:
# systematically compare 
import pandas as pd
import statsmodels.api as sm
import itertools
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

In [4]:
# Define target variables
y1 = regression_df['diff_SSP2']
y2 = np.log(regression_df['diff_SSP2'])

# Define predictor variables (excluding 'iso' and the target variable)
predictors = regression_df.columns.difference(['iso', 'diff_SSP2', 'scen_run', 'glmt_cum'])
# Generate second-order terms
poly       = PolynomialFeatures(2, interaction_only=False, include_bias=False)
X_poly     = poly.fit_transform(regression_df[predictors])
X_poly_std = StandardScaler().fit_transform(X_poly)
# Extract polynomial feature names
poly_feature_names = poly.get_feature_names_out(predictors)
predictor_df       = pd.DataFrame(X_poly_std, columns = poly_feature_names)

# interactions with tas_baseline 
interaction_terms  = [f for f in poly_feature_names if ('tas_baseline' in f) & (f != 'tas_baseline') & (f != 'tas_baseline^2')]

# Combine original predictors, interaction terms, and polynomial features 
all_predictors     = list(predictors) + list(interaction_terms) + list(predictors + '^2')

# intercept matrix for country fixed effects  
n_countries = int(len(regression_df)/1000)
X_intercept = np.zeros((len(regression_df), n_countries))
for i in range(n_countries):
    X_intercept[i*1000:(i+1)*1000, i] = 1

In [5]:
def fit_model_and_calculate_bic(X_, y_):
    model = sm.OLS(endog = y_, exog = X_).fit()
    return model.bic, model.rsquared, model

# # test function 
# X = np.c_[predictor_df.loc[:, ['tas_soc', 'tas_soc^2', 'glmt_exc', 'glmt_exc tas_baseline', 'glmt_uxc', 'glmt_uxc tas_baseline']].values, X_intercept]
# y = np.log(regression_df['diff_SSP2'])
# bic, model = fit_model_and_calculate_bic(X,y)
# print(bic)

#### combine forward and backward selection to get best model 

In [6]:
best_rsquared = 0
threshold_r   = 0
threshold_in  = 50
threshold_out = 50
included = []
best_bic = np.inf

while True:
    changed = False
    # Forward step
    excluded = list(set(all_predictors) - set(included))
    new_pval = pd.Series(index=excluded, dtype = np.float64)
    new_rval = pd.Series(index=excluded, dtype = np.float64)
    for new_column in excluded:
        X = np.c_[predictor_df[included + [new_column]], X_intercept]
        bic, rsquare, _ = fit_model_and_calculate_bic(X, y2)
        new_pval[new_column] = bic
        new_rval[new_column] = rsquare
    best_pval = new_pval.min()
    best_rval = new_rval.values[new_pval.argmin()]
    if (best_pval < best_bic - threshold_in) & (best_rval > best_rsquared + threshold_r):
        best_feature = new_pval.idxmin()
        included.append(best_feature)
        best_bic      = best_pval
        best_rsquared = best_rval
        changed       = True
        
    # Backward step
    if len(included) > 0:
        bic_with_feature = pd.Series(index=included, dtype = np.float64)
        rval_with_feature = pd.Series(index=included, dtype = np.float64)
        for col in included:
            X = np.c_[predictor_df[list(set(included) - set([col]))], X_intercept]
            bic, rsquare, _ = fit_model_and_calculate_bic(X, y2)
            bic_with_feature[col]  = bic
            rval_with_feature[col] = rsquare
        worst_pval = bic_with_feature.min()
        worst_rval = rval_with_feature.values[bic_with_feature.argmin()]
        if (worst_pval < best_bic - threshold_out) & (worst_rval > best_rsquared-threshold_r):
            worst_feature = bic_with_feature.idxmin()
            included.remove(worst_feature)
            best_bic = worst_pval
            best_bic = worst_rval
            changed = True
    
    print(best_bic)
    print(included)        
    
    if not changed:
        break
    
# this step selects ['glmt_exc tas_baseline', 'tas_soc^2', 'tas_soc', 'glmt_exc', 'glmt_uxc tas_baseline', 'glmt_uxc', 'glmt_exc^2', 'glmt_eoc^2', 'glmt_eoc', 'glmt_soc tas_baseline']
# as predictors
# all of them have statistic significance 
# glmt_max, glmt_ntwr, glmt_od are never chosen 

138392.31775647408
['glmt_exc tas_baseline']
78369.91439031047
['glmt_exc tas_baseline', 'tas_soc^2']
-91368.56041468214
['glmt_exc tas_baseline', 'tas_soc^2', 'tas_soc']
-210263.4983176071
['glmt_exc tas_baseline', 'tas_soc^2', 'tas_soc', 'glmt_exc']
-212138.16098194767
['glmt_exc tas_baseline', 'tas_soc^2', 'tas_soc', 'glmt_exc', 'glmt_uxc tas_baseline']
-213866.47424211726
['glmt_exc tas_baseline', 'tas_soc^2', 'tas_soc', 'glmt_exc', 'glmt_uxc tas_baseline', 'glmt_uxc']
-214531.3919778904
['glmt_exc tas_baseline', 'tas_soc^2', 'tas_soc', 'glmt_exc', 'glmt_uxc tas_baseline', 'glmt_uxc', 'glmt_exc^2']
-214641.92328118102
['glmt_exc tas_baseline', 'tas_soc^2', 'tas_soc', 'glmt_exc', 'glmt_uxc tas_baseline', 'glmt_uxc', 'glmt_exc^2', 'glmt_eoc^2']
-214746.70236512774
['glmt_exc tas_baseline', 'tas_soc^2', 'tas_soc', 'glmt_exc', 'glmt_uxc tas_baseline', 'glmt_uxc', 'glmt_exc^2', 'glmt_eoc^2', 'glmt_eoc']
-214746.70236512774
['glmt_exc tas_baseline', 'tas_soc^2', 'tas_soc', 'glmt_exc', 'g

In [53]:
best_rsquared = 0
threshold_r   = 0
threshold_in  = 50
threshold_out = 50
included = []
best_bic = np.inf

while True:
    changed = False
    # Forward step
    excluded = list(set(all_predictors) - set(included))
    new_pval = pd.Series(index=excluded, dtype = np.float64)
    new_rval = pd.Series(index=excluded, dtype = np.float64)
    for new_column in excluded:
        X = np.c_[predictor_df[included + [new_column]], X_intercept]
        bic, rsquare, _ = fit_model_and_calculate_bic(X, y1)
        new_pval[new_column] = bic
        new_rval[new_column] = rsquare
    best_pval = new_pval.min()
    best_rval = new_rval.values[new_pval.argmin()]
    if (best_pval < best_bic - threshold_in) & (best_rval > best_rsquared + threshold_r):
        best_feature = new_pval.idxmin()
        included.append(best_feature)
        best_bic      = best_pval
        best_rsquared = best_rval
        changed       = True
        
    # Backward step
    if len(included) > 0:
        bic_with_feature = pd.Series(index=included, dtype = np.float64)
        rval_with_feature = pd.Series(index=included, dtype = np.float64)
        for col in included:
            X = np.c_[predictor_df[list(set(included) - set([col]))], X_intercept]
            bic, rsquare, _ = fit_model_and_calculate_bic(X, y1)
            bic_with_feature[col]  = bic
            rval_with_feature[col] = rsquare
        worst_pval = bic_with_feature.min()
        worst_rval = rval_with_feature.values[bic_with_feature.argmin()]
        if (worst_pval < best_bic - threshold_out) & (worst_rval > best_rsquared-threshold_r):
            worst_feature = bic_with_feature.idxmin()
            included.remove(worst_feature)
            best_bic = worst_pval
            best_bic = worst_rval
            changed = True
    
    print(best_bic)
    print(included)        
    
    if not changed:
        break
    
# this step selects ['glmt_exc tas_baseline', 'tas_soc^2', 'tas_soc', 'glmt_exc', 'glmt_uxc tas_baseline', 'glmt_uxc', 'glmt_exc^2', 'glmt_eoc^2', 'glmt_eoc', 'glmt_soc tas_baseline']
# as predictors
# all of them have statistic significance 
# glmt_max, glmt_ntwr, glmt_od are never chosen 

263918.02418369957
['glmt_exc tas_baseline']
252798.1554043519
['glmt_exc tas_baseline', 'glmt_exc']
245998.53409312846
['glmt_exc tas_baseline', 'glmt_exc', 'tas_soc^2']
159150.93094773847
['glmt_exc tas_baseline', 'glmt_exc', 'tas_soc^2', 'tas_soc']
145590.15456262918
['glmt_exc tas_baseline', 'glmt_exc', 'tas_soc^2', 'tas_soc', 'tas_baseline tas_soc']
144401.6354399614
['glmt_exc tas_baseline', 'glmt_exc', 'tas_soc^2', 'tas_soc', 'tas_baseline tas_soc', 'glmt_uxc tas_baseline']
144050.5610097113
['glmt_exc tas_baseline', 'glmt_exc', 'tas_soc^2', 'tas_soc', 'tas_baseline tas_soc', 'glmt_uxc tas_baseline', 'glmt_exc^2']
143730.68404697423
['glmt_exc tas_baseline', 'glmt_exc', 'tas_soc^2', 'tas_soc', 'tas_baseline tas_soc', 'glmt_uxc tas_baseline', 'glmt_exc^2', 'glmt_uxc']
143730.68404697423
['glmt_exc tas_baseline', 'glmt_exc', 'tas_soc^2', 'tas_soc', 'tas_baseline tas_soc', 'glmt_uxc tas_baseline', 'glmt_exc^2', 'glmt_uxc']


In [6]:
sel_predictors = ['glmt_exc tas_baseline', 'tas_soc^2', 'tas_soc', 'glmt_exc', 'glmt_uxc tas_baseline', 'glmt_uxc', 'glmt_exc^2', 'glmt_eoc^2', 'glmt_eoc']
y              = np.log(regression_df['diff_SSP2'])
bic, r, model  = fit_model_and_calculate_bic(np.c_[predictor_df[sel_predictors], X_intercept], y)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              diff_SSP2   R-squared:                       0.957
Model:                            OLS   Adj. R-squared:                  0.957
Method:                 Least Squares   F-statistic:                 2.109e+04
Date:                Mon, 17 Jun 2024   Prob (F-statistic):               0.00
Time:                        09:28:56   Log-Likelihood:             1.0837e+05
No. Observations:              157000   AIC:                        -2.164e+05
Df Residuals:                  156834   BIC:                        -2.147e+05
Df Model:                         165                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.6404      0.001   -580.559      0.000      -0.643      -0.638
x2            10.9831      0.010   1048.966      0.000      10.963      11.004
x3            -7.9824      0.010   -823.039      0.000      -8.001      -7.963
x4             0.3559      0.002    161.665      0.000       0.352       0.360
x5             0.0568      0.001     55.037      0.000       0.055       0.059
x6            -0.0354      0.001    -34.814      0.000      -0.037      -0.033
x7            -0.0120      0.002     -7.336      0.000      -0.015      -0.009
x8            -0.1790      0.016    -11.301      0.000      -0.210      -0.148
x9             0.1743      0.016     10.801      0.000       0.143       0.206
x10            3.6737      0.006    609.797      0.000       3.662       3.686
x11           -0.4060      0.004    -91.972      0.000      -0.415      -0.397
x12            3.6077      0.006    627.300      0.000       3.596       3.619
x13           -6.4666      0.008   -797.717      0.000      -6.482      -6.451
x14            2.9359      0.005    602.865      0.000       2.926       2.945
x15            3.3266      0.007    451.333      0.000       3.312       3.341
x16            3.2760      0.005    631.990      0.000       3.266       3.286
x17            3.0279      0.008    392.714      0.000       3.013       3.043
x18            3.2217      0.007    430.762      0.000       3.207       3.236
x19            0.1951      0.004     45.445      0.000       0.187       0.203
x20            3.5488      0.007    513.335      0.000       3.535       3.562
x21           -4.8290      0.007   -687.358      0.000      -4.843      -4.815
x22           -6.7429      0.008   -800.803      0.000      -6.759      -6.726
x23           -3.3141      0.006   -556.033      0.000      -3.326      -3.302
x24            3.7507      0.006    595.362      0.000       3.738       3.763
x25            3.7332      0.006    606.720      0.000       3.721       3.745
x26            2.7000      0.008    332.894      0.000       2.684       2.716
x27           -3.6705      0.006   -592.050      0.000      -3.683      -3.658
x28            1.5208      0.004    352.611      0.000       1.512       1.529
x29           -1.0070      0.005   -217.074      0.000      -1.016      -0.998
x30           -4.7360      0.007   -676.953      0.000      -4.750      -4.722
x31            0.5871      0.004    138.006      0.000       0.579       0.595
x32            0.0260      0.004      6.063      0.000       0.018       0.034
x33           -2.8491      0.006   -506.922      0.000      -2.860      -2.838
x34            1.8898      0.009    210.708      0.000       1.872       1.907
x35            2.7823      0.008    348.171      0.000       2.767       2.798
x36            3.2269      0.007    432.267      0.000       3.212       3.242
x3

In [8]:
result_df                  = pd.DataFrame(X_poly, columns = poly_feature_names)[sel_predictors]
result_df['log_diff_SSP2'] = np.log(regression_df['diff_SSP2'])
result_df['iso']           = regression_df['iso']
result_df['scen_run']      = regression_df['scen_run']

In [9]:
cset.path_CHAR_results.mkdir(parents=True, exist_ok=True)
result_df.to_csv(Path.joinpath(cset.path_CHAR_results, 'regression_dataset.csv'))